In [ ]:
# Minimize chance of race condition by loading config first
import omegaconf
# cfg = omegaconf.OmegaConf.load('mt50_success_plots.debug.yaml')
cfg = omegaconf.OmegaConf.load('mt50_success_plots.yaml')
cfg['in_files'], cfg['out_files'];

In [ ]:
import glob
from typing import Dict, List, Tuple
import json
import re

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from itertools import cycle
import matplotlib
matplotlib.rcParams.update(
    {
        "figure.dpi": 150,
        "font.size": 12,
    }
)
matplotlib.rcParams["pdf.fonttype"] = 42
matplotlib.rcParams["ps.fonttype"] = 42

In [ ]:
in_files = {}
for pairs in cfg['in_files']:
    for (k, in_pat) in pairs.items():
        if in_pat.strip().startswith('/'):
            in_files.setdefault(k, []).extend(glob.glob(in_pat))
        else:
            in_files.setdefault(k, []).extend(glob.glob(f'../{in_pat}'))

In [ ]:
in_files;

In [ ]:
CONTENT = ""
def merge_result_files(result_filenames: List[str], last_t=False) -> Dict[str, List[float]]:
    global CONTENT
    perf = {}
    for filename in result_filenames:
        with open(filename) as f:
            if filename.endswith(".json"):
                data = json.load(f)
            elif filename.endswith(".ndjson"):
                content = f.read()
                CONTENT = content
                objects = re.split(r"\n(?=\{)", content.strip())
                if last_t:
                    data = json.loads(objects[-1])
                else:
                    # datums = [json.loads(obj) for obj in objects]
                    data = {}
                    for d in [json.loads(obj) for obj in objects]:
                        for k, v in d.items():
                            if k not in data:
                                data[k] = [v]
                            else:
                                data[k].append(v)
            else:
                warnings.warn(f"Could not load file: {filename}")
                data = {}
        for k, v in data.items():
            if k not in perf:
                perf[k] = [v]
            else:
                perf[k].append(v)
    return perf

In [ ]:
results = {k: merge_result_files(v) for (k, v) in in_files.items()}

In [ ]:
results;

In [ ]:
def cumulative_dist(values):
    xs = []
    ys = []
    sorted_vals = sorted(values, reverse=True)
    current_val = sorted_vals[0]
    y = 0
    for i, v in enumerate(sorted_vals):
        y += 1
        if current_val != v:
            if not ys:
                xs.append(v)
                ys.append(0)
            else:
                xs.append(v)
                ys.append(ys[-1])
            xs.append(v)
            ys.append(y)
            current_val = v
    xs.append(v)
    ys.append(y)
    return xs, ys

In [ ]:

x_results = {}
y_results = {}
entries = []
success_rate_interp = np.linspace(0., 1.0, 101)
num_tasks_interp = np.array([x 
                             for x0 in range(51)
                             for x in [x0 - 1e-6, x0]
                             ])
for result_key, values in results.items():
    x_results[result_key] = []
    y_results[result_key] = []
    for i in range(4):
        if 'assembly/SuccessRate' in values:
            xs, ys = cumulative_dist([max(vs[i][-10:])
                                      for (k, vs) in values.items() 
                                      if 'SuccessRate' in k and vs[i]])
            missing_k = [k for (k, vs) in values.items() if 'SuccessRate' in k and not vs[i]]
            for missed_k in missing_k:
                print(f"Missed: {missed_k} in result_key {result_key}")
        elif isinstance(next(iter(values.values()))[0], float):
            xs, ys = cumulative_dist([vs[i] for vs in values.values()])
        else:
            raise NotImplementedError()
        # x_results[result_key].append(xs)
        # y_results[result_key].append(ys)
        xi = np.interp(num_tasks_interp, ys, xs)
        yi = num_tasks_interp
        x_results[result_key].append(xi)
        y_results[result_key].append(yi)
        # fig, ax = plt.subplots()
        # plt.title(f"{result_key} run {i}")
        # ax.plot(ys, xs)
        # ax.plot(yi, xi)
        # ax.invert_xaxis()
        # fig.show()
        for (x, y) in zip(xi, yi):
            entries.append({
                "Method": result_key,
                "Success Rate": x,
                "Number of Tasks": y,
                "Seed": i,
            })
    # ys = y_results[result_key][0]
    # x_stack = np.stack(x_results[result_key], axis=0)
    # fig, ax = plt.subplots()
    # plt.title(f"{result_key} summary")
    # ax.plot(y_results[result_key][0], xs)
    # ax.plot(ys, np.min(x_stack, axis=0))
    # ax.plot(ys, np.max(x_stack, axis=0))
    # ax.fill_between(ys, np.min(x_stack, axis=0), np.max(x_stack, axis=0))
    # ax.plot(ys, np.mean(x_stack, axis=0))
    # ax.invert_xaxis()
    # fig.show()

lines = ["-","--","-.",":"]
linecycler = cycle(lines)
plt.style.use("seaborn-v0_8-colorblind")
fig, ax = plt.subplots()
plt.title(cfg['title'])
plt.xlabel('Number of Tasks')
plt.ylabel('Success Rate')
legend_lines = []
legend_labels = []
for result_key, xs in x_results.items():
    ys = y_results[result_key][0]
    x_stack = np.stack(xs, axis=0)
    # ax.plot(y_results[result_key][0], xs)
    # ax.plot(ys, np.min(x_stack, axis=0))
    # ax.plot(ys, np.max(x_stack, axis=0))
    fill = ax.fill_between(ys, np.min(x_stack, axis=0), np.max(x_stack, axis=0), 
                    label=result_key, alpha=0.2)
    line = ax.plot(ys, np.mean(x_stack, axis=0), label=result_key,
                   linestyle=next(linecycler))
    legend_lines.append((fill, line[0]))
    legend_labels.append(result_key)
plt.legend(legend_lines, legend_labels)
ax.invert_xaxis()
for out_f in cfg['out_files']:
    fig.savefig(out_f)